# Parser for Open Image's Class Hierarchy
Assumption: Only focus on the read performance. Write performance and memory/storage is not a big concern.
- Hierarchy file is small
- Access pattern mostly likely supporting above operation
- Like write-once, read-many during the whole training and inferencing lifecycle

There are two proposed solutions for high read performance under python.

**First Solution:**
Parse the JSON into bi-directional linked list. And compute the required result on-the-fire.

PRO: lower memory.

CONS: a bit slower the second one

**Second Solution:**
Parse the JSON into bi-directional linked list. And built mapping caching layer for the required result directly.

Higher: Super fast in read performance

CONS: Taker more time in the parsing step (Write) and consuming more memory

*Remark:*
Code written in lower-level language (e.g. C, rust) will be certainly more efficient if that is mission critical function.

# Initial data mapping loading

In [156]:
import json
import csv
import os
import pandas as pd

data_path = '../data'
class_description_file = os.path.join(data_path, 'oidv6-class-descriptions.csv')
label_hierarchy = os.path.join(data_path, 'bbox_labels_600_hierarchy.json')
id_to_name_df = pd.read_csv(class_description_file, index_col='LabelName')
name_to_id_df = pd.read_csv(class_description_file, index_col='DisplayName')

def id_to_name(LabelName: str):
    try:
        displayName = id_to_name_df.loc[LabelName]['DisplayName']
    except KeyError:
        print(f"ERROR: {LabelName}'s Display name not found'.")
        displayName = "Undefined"
    return displayName

def name_to_id(DisplayName: str):
    try:
        labelName = name_to_id_df.loc[DisplayName]['LabelName']
    except KeyError:
        print(f"ERROR: {DisplayName}'s Label not found'.")
        labelName = "/x/xxxxx"
    return labelName

## Test of the Name & id mapping

In [7]:
# Test of getting namd to id
name_to_id('Dog')

'/m/0bt9lr'

In [154]:
# Test of getting id to name
id_to_name('/m/02vwcm')


'Whisk'

In [157]:
# Name of the main node
id_to_name('/m/0bl9f')

ERROR: /m/0bl9f's Display name not found'.


'Undefined'

# The double-referencing Link tree stucture

In [158]:
class HierarchyNode:
    
    id_to_node = {}
    name_to_node = {}

    @classmethod 
    def update_id_map(cls, id, node):
        cls.id_to_node[id] = node

    @classmethod 
    def update_name_map(cls, name, node):
        cls.name_to_node[name] = node

    @classmethod 
    def get_node_by_id(cls, id):
        return cls.id_to_node[id]

    @classmethod 
    def get_node_by_name(cls, name):
        return cls.name_to_node[name]

    @classmethod
    def get_siblings_name_by_name(cls, name):
        node = cls.get_node_by_name(name)
        sibling_list = node.parent.get_child()
        sibling_list.remove(node)
        return [x.name for x in sibling_list]  

    @classmethod
    def get_parent_name_by_id(cls, name):
        node = cls.get_node_by_id(name)
        if node.parent is not None:
            return node.parent.name
        else:
            return None

    @classmethod
    def get_parent_name_by_name(cls, name):
        node = cls.get_node_by_name(name)
        if node.parent is not None:
            return node.parent.name
        else:
            return None

    @classmethod
    def get_siblings_name_by_name(cls, name):
        node = cls.get_node_by_name(name)
        sibling_list = node.parent.get_child()
        sibling_list.remove(node)
        return [x.name for x in sibling_list]  

    @classmethod
    def get_siblings_name_by_id(cls, id):
        node = cls.get_node_by_id(id)
        sibling_list = node.parent.get_child()
        sibling_list.remove(node)
        return [x.name for x in sibling_list]  

    @classmethod
    def get_ancestor_name_by_id(cls, id):
        node = cls.get_node_by_id(id)
        return [x.name for x in node.get_ancestor()]  

    @classmethod
    def get_ancestor_name_by_name(cls, name):
        node = cls.get_node_by_name(name)
        return [x.name for x in node.get_ancestor()]  

    @classmethod
    def get_ancestor_name_by_name(cls, name):
        node = cls.get_node_by_name(name)
        return [x.name for x in node.get_ancestor()]  

    @classmethod
    def is_same_ancestor(cls, class_name_1, class_name_2):
        node1 = cls.get_node_by_name(class_name_1)
        node2 = cls.get_node_by_name(class_name_2)
        return node1.get_root().id == node2.get_root().id

    def __init__(self, id, parent=None):
        self.id = id
        self.parent = parent
        self.subcategory = []
        self.part = []
        self.update_id_map(id, self)
        self.name = id_to_name(id)
        self.update_name_map(self.name, self)

    def add_subcategory(self, child_node):
        self.subcategory.append(child_node)

    def add_part(self, child_node):
        self.part.append(child_node)
    
    def get_parent(self):
        return self.parent
    
    def get_ancestor(self):
        if self.parent is None:
            return []
        else:
            return  self.parent.get_ancestor() + [self.parent]

    def get_child(self):
        return self.part + self.subcategory
 
    def get_root(self):
        if self.parent is None:
            return self
        else:
            return self.parent.get_root()



# Parser

In [159]:
def node_parser(json_object, parent=None):
    if 'LabelName' in json_object:
        node = HierarchyNode(json_object['LabelName'], parent)
        if 'Subcategory' in json_object and 'LabelName' in json_object:
            for subcategory in json_object['Subcategory']:
                child = node_parser(subcategory, node)
                node.add_subcategory(child)
        if 'Part' in json_object and 'LabelName' in json_object:
            for part in json_object['Part']:
                child = node_parser(part, node)
                node.add_part(child)
        return node
    else:
        print(json_object)
        return None

# Execute the parser
## Exclude the highest level node as that is the "entity"

In [160]:
with open(label_hierarchy) as json_file:  
    json_object = json.load(json_file)
    if 'Subcategory' in json_object and 'LabelName' in json_object:
        for subcategory in json_object['Subcategory']:
            child = node_parser(subcategory, None)


# Sample Call for the Hierarchy Node Class Method

In [162]:
# print(HierarchyNode.name_to_node.keys())

# Find all siblings class of a class name
print(HierarchyNode.get_siblings_name_by_name('Can opener'))
print(HierarchyNode.get_siblings_name_by_id('/m/027rl48'))


# Find the parent class of a class name
print(HierarchyNode.get_parent_name_by_name('Can opener'))
print(HierarchyNode.get_parent_name_by_id('/m/027rl48'))

# Find all ancestor classes of a class name
print(HierarchyNode.get_ancestor_name_by_name('Can opener'))
print(HierarchyNode.get_ancestor_name_by_id('/m/027rl48'))

# Find if both class 1 and class 2 belong to the same ancestor class
## This fucntion assuming the requirement is that two classes are same if the root node is the same. i.e. Tool and Kitchen utensil are same.
## If this assumption is not true, it can be just modified
print(HierarchyNode.is_same_ancestor('Chopsticks', 'Spatula'))
print(HierarchyNode.is_same_ancestor('Chopsticks', 'Toy'))

['Chopsticks', 'Ladle', 'Spatula', 'Cutting board', 'Whisk', 'Drinking straw', 'Knife', 'Bottle opener', 'Measuring cup', 'Pizza cutter', 'Spoon', 'Fork']
['Chopsticks', 'Spatula', 'Can opener', 'Cutting board', 'Whisk', 'Drinking straw', 'Knife', 'Bottle opener', 'Measuring cup', 'Pizza cutter', 'Spoon', 'Fork']
Kitchen utensil
Kitchen utensil
['Tool', 'Kitchen utensil']
['Tool', 'Kitchen utensil']
True
False
